In [2]:
import cv2
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
import math


In [ ]:
img = cv2.imread("media/fotos/1t.jpg")
if img is None:
    print('No hay foto ')
else:
    cv2.imshow('Image', img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [ ]:
from mediapipe import solutions
from mediapipe.framework.formats import landmark_pb2
import numpy as np

MARGIN = 10  # pixels
FONT_SIZE = 1
FONT_THICKNESS = 1
HANDEDNESS_TEXT_COLOR = (88, 205, 54) # vibrant green

def draw_landmarks_on_image(rgb_image, detection_result):
  hand_landmarks_list = detection_result.hand_landmarks
  handedness_list = detection_result.handedness
  annotated_image = np.copy(rgb_image)

  # Loop through the detected hands to visualize.
  for idx in range(len(hand_landmarks_list)):
    hand_landmarks = hand_landmarks_list[idx]
    handedness = handedness_list[idx]

    # Draw the hand landmarks.
    hand_landmarks_proto = landmark_pb2.NormalizedLandmarkList()
    hand_landmarks_proto.landmark.extend([
      landmark_pb2.NormalizedLandmark(x=landmark.x, y=landmark.y, z=landmark.z) for landmark in hand_landmarks
    ])
    solutions.drawing_utils.draw_landmarks(
      annotated_image,
      hand_landmarks_proto,
      solutions.hands.HAND_CONNECTIONS,
      solutions.drawing_styles.get_default_hand_landmarks_style(),
      solutions.drawing_styles.get_default_hand_connections_style())

    # Get the top left corner of the detected hand's bounding box.
    height, width, _ = annotated_image.shape
    x_coordinates = [landmark.x for landmark in hand_landmarks]
    y_coordinates = [landmark.y for landmark in hand_landmarks]
    text_x = int(min(x_coordinates) * width)
    text_y = int(min(y_coordinates) * height) - MARGIN

    # Draw handedness (left or right hand) on the image.
    cv2.putText(annotated_image, f"{handedness[0].category_name}",
                (text_x, text_y), cv2.FONT_HERSHEY_DUPLEX,
                FONT_SIZE, HANDEDNESS_TEXT_COLOR, FONT_THICKNESS, cv2.LINE_AA)

  return annotated_image

In [ ]:
mp_image = mp.Image.create_from_file('media/fotos/1t.jpg')
model_path = 'hand_landmarker.task'

BaseOptions = mp.tasks.BaseOptions
HandLandmarker = mp.tasks.vision.HandLandmarker
HandLandmarkerOptions = mp.tasks.vision.HandLandmarkerOptions
VisionRunningMode = mp.tasks.vision.RunningMode

# Create a hand landmarker instance with the image mode:
options = HandLandmarkerOptions(
    base_options=BaseOptions(model_asset_path='hand_landmarker.task'),
    running_mode=VisionRunningMode.IMAGE)

In [ ]:

with HandLandmarker.create_from_options(options) as landmarker:
  # The landmarker is initialized. Use it here.
  # ...
    hand_landmarker_result = landmarker.detect(mp_image)
    
    landmarks = hand_landmarker_result.hand_landmarks
    if landmarks:
      for hand_landmarks in landmarks:
          thumb_tip = hand_landmarks[4]  # Índice 4 para la punta del pulgar
          index_tip = hand_landmarks[8]  # Índice 8 para la punta del índice

          thumb_x, thumb_y, thumb_z = thumb_tip.x, thumb_tip.y, thumb_tip.z
          index_x, index_y, index_z = index_tip.x, index_tip.y, index_tip.z

          # Calcular la distancia entre la punta del pulgar y la del índice
          distance = ((thumb_x - index_x) ** 2 + (thumb_y - index_y) ** 2 + (thumb_z - index_z) ** 2) ** 0.5

          # Establecer un umbral para considerar que están juntos
          threshold = 0.06

          if distance < threshold:
              print("La punta del pulgar y la del índice están juntas.")
          else:
              print("La punta del pulgar y la del índice están separadas.")
          print(hand_landmarks[0])
    else:
        print("no se detecta la mano :(")

    

    
    annotated_image = draw_landmarks_on_image(mp_image.numpy_view(), hand_landmarker_result)


    cv2.imshow('Image', cv2.cvtColor(annotated_image, cv2.COLOR_RGB2BGR))
    cv2.waitKey(0)
    cv2.destroyAllWindows()
  
    

In [ ]:

cap = cv2.VideoCapture(0)
model_path = 'hand_landmarker.task'
BaseOptions = mp.tasks.BaseOptions
HandLandmarker = mp.tasks.vision.HandLandmarker
HandLandmarkerOptions = mp.tasks.vision.HandLandmarkerOptions
HandLandmarkerResult = mp.tasks.vision.HandLandmarkerResult
VisionRunningMode = mp.tasks.vision.RunningMode

def print_result():
    

options = HandLandmarkerOptions(
    base_options=BaseOptions(model_asset_path='/path/to/model.task'),
    running_mode=VisionRunningMode.LIVE_STREAM,
    result_callback=print_result)



In [12]:
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands


cap = cv2.VideoCapture(0)
with mp_hands.Hands(
    model_complexity=0,
    min_detection_confidence=0.5,
    #max_num_hands=2,
    min_tracking_confidence=0.5) as hands:
  

  #aqui guardaremos los circulos que vamos dibujando  
  previous_positions = [[]]
  positions_index = 0

  unido = False

  while cap.isOpened():
    success, image = cap.read()
    if not success:
      print("Ignoring empty camera frame.")
      # If loading a video, use 'break' instead of 'continue'.
      continue


                     
    # To improve performance, optionally mark the image as not writeable to
    # pass by reference.
    image.flags.writeable = False
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)   
    results = hands.process(image)

    # Draw the hand annotations on the image.
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    if results.multi_hand_landmarks:
      for hand_landmarks in results.multi_hand_landmarks:
        
        mp_drawing.draw_landmarks(
            image,
            hand_landmarks,
            mp_hands.HAND_CONNECTIONS,
            mp_drawing_styles.get_default_hand_landmarks_style(),
            mp_drawing_styles.get_default_hand_connections_style())
        #print(hand_landmarks)
        
        
        thumb_x = hand_landmarks.landmark[4].x 
        thumb_y = hand_landmarks.landmark[4].y
        thumb_z = hand_landmarks.landmark[4].z 

        index_x = hand_landmarks.landmark[8].x
        index_y = hand_landmarks.landmark[8].y 
        index_z = hand_landmarks.landmark[8].z

        #print("pulgar:/nx: ", thumb_x, "/ny: ", thumb_y, "/nz: ", thumb_z)
         

        
        
        # Calcular la distancia entre la punta del pulgar y la del índice
      distance = math.sqrt((thumb_x - index_x)**2 + (thumb_y - index_y)**2)# + (thumb_z - index_z)**2)   
      #print(distance)

          # Establecer un umbral para considerar que están juntos
      threshold = 0.07

      if distance < threshold:
          unido = True
          midpoint = ((thumb_x + index_x) / 2, (thumb_y + index_y) / 2, (thumb_z + index_z) / 2)
          previous_positions[positions_index].append(midpoint)
      else:
        if unido == True:
          unido = False
          positions_index += 1
          previous_positions.append([])




    #x, y, z = (0, 0, 0)
    for position in previous_positions:
      if positions_index < len(position):
        for pos in position:
          
          #print(position[positions_index])
          x, y, z = pos
        
          height, width, deep = image.shape
          midpoint_x, midpoint_y, midpoint_z= int(x * width), int(y * height), int(z * deep)
          prev_midpoint_x, prev_midpoint_y, prev_midpoint_z= int(prev_x * width), int(prev_y * height), int(prev_z * deep)
          cv2.line(image, (midpoint_x, midpoint_y), (prev_midpoint_x, prev_midpoint_y), (0, 255, 0), 5)      
        prev_x, prev_y, prev_z = pos                                                         
      
          
      #cv2.circle(image, (midpoint_x, midpoint_y), 3, (0, 255, 0), -1)
          
      
        
    
    # Flip the image horizontally for a selfie-view display.
    cv2.imshow('MediaPipe Hands', cv2.flip(image, 1))
    if cv2.waitKey(1) & 0xFF == ord('q'):
      break
cap.release()
cv2.destroyAllWindows()



In [9]:
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands


cap = cv2.VideoCapture(0)
with mp_hands.Hands(
    model_complexity=0,
    min_detection_confidence=0.5,
    #max_num_hands=2,
    min_tracking_confidence=0.5) as hands:
  

  #aqui guardaremos los circulos que vamos dibujando  
  previous_positions = []
  positions_index = 0

  unido = False

  while cap.isOpened():
    success, image = cap.read()
    if not success:
      print("Ignoring empty camera frame.")
      # If loading a video, use 'break' instead of 'continue'.
      continue


                     
    # To improve performance, optionally mark the image as not writeable to
    # pass by reference.
    image.flags.writeable = False
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)   
    results = hands.process(image)

    # Draw the hand annotations on the image.
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    if results.multi_hand_landmarks:
      for hand_landmarks in results.multi_hand_landmarks:
        
        mp_drawing.draw_landmarks(
            image,
            hand_landmarks,
            mp_hands.HAND_CONNECTIONS,
            mp_drawing_styles.get_default_hand_landmarks_style(),
            mp_drawing_styles.get_default_hand_connections_style())
        #print(hand_landmarks)
        
        
        thumb_x = hand_landmarks.landmark[4].x 
        thumb_y = hand_landmarks.landmark[4].y
        thumb_z = hand_landmarks.landmark[4].z 

        index_x = hand_landmarks.landmark[8].x
        index_y = hand_landmarks.landmark[8].y 
        index_z = hand_landmarks.landmark[8].z

        #print("pulgar:/nx: ", thumb_x, "/ny: ", thumb_y, "/nz: ", thumb_z)
         

        
        
        # Calcular la distancia entre la punta del pulgar y la del índice
      distance = math.sqrt((thumb_x - index_x)**2 + (thumb_y - index_y)**2)# + (thumb_z - index_z)**2)   
      #print(distance)

          # Establecer un umbral para considerar que están juntos
      threshold = 0.07

      if distance < threshold:
          
          midpoint = ((thumb_x + index_x) / 2, (thumb_y + index_y) / 2, (thumb_z + index_z) / 2)
          previous_positions.append(midpoint)

      """
      else:
        if unido == True:
          unido = False
          positions_index += 1
          previous_positions.append([])
      
      """




    x, y, z = (0, 0, 0)
    for position in previous_positions:
        prev_x, prev_y, prev_z = x, y, z
        x, y, z = position
        
        height, width, deep = image.shape
        midpoint_x, midpoint_y, midpoint_z= int(x * width), int(y * height), int(z * deep)
        prev_midpoint_x, prev_midpoint_y, prev_midpoint_z= int(prev_x * width), int(prev_y * height), int(prev_z * deep)                                                               
      
          
        #cv2.circle(image, (midpoint_x, midpoint_y), 3, (0, 255, 0), -1)
        cv2.line(image, (midpoint_x, midpoint_y), (prev_midpoint_x, prev_midpoint_y), (0, 255, 0), 5)
      
        
    
    # Flip the image horizontally for a selfie-view display.
    cv2.imshow('MediaPipe Hands', cv2.flip(image, 1))
    if cv2.waitKey(1) & 0xFF == ord('q'):
      break
cap.release()
cv2.destroyAllWindows()

